[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Training_Tutorial.ipynb)

## Create the models
Import the models from the ISR package and create

- a RRDN super scaling network
- a discriminator network for GANs training
- a VGG19 feature extractor to train with a perceptual loss function

Carefully select
- 'x': this is the upscaling factor (2 by default)
- 'layers_to_extract': these are the layers from the VGG19 that will be used in the perceptual loss (leave the default if you're not familiar with it)
- 'lr_patch_size': this is the size of the patches that will be extracted from the LR images and fed to the ISR network during training time

Play around with the other architecture parameters

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
# import tensorflow as tf
# # 查看gpu和cpu的数量
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
# print("Num GPUs Available: ", len((tf.config.experimental.list_physical_devices('GPU'))))
# print(gpus, cpus)

In [3]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

In [4]:

# import torch
# print(torch.__version__)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #返回cuda表示成功
# #或者
# print(torch.cuda.is_available())
# #返回True表示成功

In [5]:
print(os.getcwd())

d:\Master_Thesis\image-super-resolution-master\notebooks


In [6]:
lr_train_patch_size = 40#40
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

In [7]:
print(os.getcwd())

d:\Master_Thesis\image-super-resolution-master\notebooks


## Give the models to the Trainer
The Trainer object will combine the networks, manage your training data and keep you up-to-date with the training progress through Tensorboard and the command line.

Here we do not use  the pixel-wise MSE but only the perceptual loss by specifying the respective weights in `loss_weights`

In [8]:
from ISR.train import Trainer
loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

path = os.getcwd()#d:/Master_Thesis/image-super-resolution-master/notebooks
log_dirs = {'logs': path + '/ISR_chr/logs', 'weights': path + '/ISR_chr/weights'}

learning_rate = {'initial_value': 0.001, 'decay_factor': 0.5, 'decay_frequency': 30}
#learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}
flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    # lr_train_dir=path + '/ISR_chr/data/train_LR/',
    # hr_train_dir=path + '/ISR_chr/data/train_HR_24/',
    # lr_valid_dir=path + '/ISR_chr/data/valid_LR/',
    # hr_valid_dir=path + '/ISR_chr/data/valid_HR_2/',
    # lr_train_dir=path + '/ISR_chr/data/final_data/data_aug/train/',
    # hr_train_dir=path + '/ISR_chr/data/final_data/labels_aug/train/',
    # lr_valid_dir=path + '/ISR_chr/data/final_data/data_aug/val/',
    #hr_valid_dir=path + '/ISR_chr/data/final_data/labels_aug/val/',

    # lr_train_dir=path + '/ISR_chr/data/100_70/final2/bw70/train/',
    # lr_valid_dir=path + '/ISR_chr/data/100_70/final2/bw70/val/',

    # hr_train_dir=path + '/ISR_chr/data/100_70/final2/bw100_x2/train/',
    # hr_valid_dir=path + '/ISR_chr/data/100_70/final2/bw100_x2/val/',
    lr_train_dir=path + '/ISR_chr/data/all_data/Undersampling_ISR/data/train/',
    lr_valid_dir=path + '/ISR_chr/data/all_data/Undersampling_ISR/data/val/',

    hr_train_dir=path + '/ISR_chr/data/all_data/Undersampling_ISR/label/train/',
    hr_valid_dir=path + '/ISR_chr/data/all_data/Undersampling_ISR/label/val/',

    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2k',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=200,#33,#40
)


Choose epoch number, steps and batch size and start training

In [9]:
trainer.train(
    epochs=100,#10
    steps_per_epoch=20,#20
    batch_size=4,#4
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)


Training details:
  training_parameters: 
    lr_train_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/Undersampling_ISR/data/train/
    hr_train_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/Undersampling_ISR/label/train/
    lr_valid_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/Undersampling_ISR/data/val/
    hr_valid_dir: d:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/data/all_data/Undersampling_ISR/label/val/
    loss_weights: {'generator': 0.0, 'feature_extractor': 0.0833, 'discriminator': 0.01}
    log_dirs: {'logs': 'd:\\Master_Thesis\\image-super-resolution-master\\notebooks/ISR_chr/logs', 'weights': 'd:\\Master_Thesis\\image-super-resolution-master\\notebooks/ISR_chr/weights'}
    fallback_save_every_n_epochs: 2
    dataname: div2k
    n_validation: 200
    flatness: {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}
    learning

Epoch 0/100
Current learning rate: 0.0010000000474974513
100%|██████████| 20/20 [02:26<00:00,  7.34s/it]
Epoch 0 took      146.9s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.3143043677136302, 'val_generator_loss': 0.46979675, 'val_discriminator_loss': 0.016982388, 'val_feature_extractor_loss': 1.2027937, 'val_feature_extractor_1_loss': 6.3394527, 'val_generator_PSNR_Y': 7.3854194, 'train_d_real_loss': 0.064031236, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.019656, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.09858669, 'train_generator_loss': 0.3351476, 'train_discriminator_loss': 0.022539793, 'train_feature_extractor_loss': 0.42523196, 'train_feature_extractor_1_loss': 1.9363837, 'train_generator_PSNR_Y': 10.760466}
val_generator_PSNR_Y improved from       -inf to    7.38542
Saving weights
Epoch 1/100
Current learning rate: 0.0010000000474974513
100%|██████████| 20/20 [02:24<00:00,  7.22s/it]
Epoch 1 took      144.5s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

{'val_loss': 0.07167635224759579, 'val_generator_loss': 0.25607988, 'val_discriminator_loss': 2.357587, 'val_feature_extractor_loss': 0.20207652, 'val_feature_extractor_1_loss': 0.95279694, 'val_generator_PSNR_Y': 21.87405, 'train_d_real_loss': 0.00015527468, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.00013656107, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.031563967, 'train_generator_loss': 0.30896795, 'train_discriminator_loss': 1.630048, 'train_feature_extractor_loss': 0.09134377, 'train_feature_extractor_1_loss': 0.27512646, 'train_generator_PSNR_Y': 21.345022}
val_generator_PSNR_Y did not improve.
Saving weights anyways.
Epoch 98/100
Current learning rate: 0.0001250000059371814
Current flatness treshold: 0.15
100%|██████████| 20/20 [02:23<00:00,  7.17s/it]
Epoch 98 took      143.3s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.06866052342578768, 'val_generator_loss': 0.2670554, 'val_discriminator_loss': 2.3055182, 'val_feature_extractor_loss': 0.19556324, 'val_feature_extractor_1_loss': 0.8994028, 'val_generator_PSNR_Y': 21.557446, 'train_d_real_loss': 0.00017271415, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.00014231134, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.040207386, 'train_generator_loss': 0.3123829, 'train_discriminator_loss': 2.0338748, 'train_feature_extractor_loss': 0.120336115, 'train_feature_extractor_1_loss': 0.35670194, 'train_generator_PSNR_Y': 21.042536}
val_generator_PSNR_Y did not improve.
Epoch 99/100
Current learning rate: 0.0001250000059371814
Current flatness treshold: 0.15
100%|██████████| 20/20 [02:23<00:00,  7.19s/it]
Epoch 99 took      143.7s


800/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.0762210151180625, 'val_generator_loss': 0.2843191, 'val_discriminator_loss': 2.3056839, 'val_feature_extractor_loss': 0.21384697, 'val_feature_extractor_1_loss': 1.0626035, 'val_generator_PSNR_Y': 20.868315, 'train_d_real_loss': 0.00016524138, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 9.422693e-05, 'train_d_fake_accuracy': 1.0, 'train_loss': 0.040791005, 'train_generator_loss': 0.31360802, 'train_discriminator_loss': 1.815155, 'train_feature_extractor_loss': 0.106277086, 'train_feature_extractor_1_loss': 0.43728733, 'train_generator_PSNR_Y': 20.835745}
val_generator_PSNR_Y did not improve.


In [10]:
%load_ext tensorboard

In [11]:
#%tensorboard --logdirs= 'D:\Master_Thesis\image-super-resolution-master\notebooks/ISR_chr/logs/rrdn-C4-D3-G64-G064-T10-x2/2024-03-15_1048/'
%tensorboard --logdirs= 'D:/Master_Thesis/image-super-resolution-master/notebooks/ISR_chr/logs/rrdn-C4-D3-G64-G064-T10-x2/2024-03-15_1048/'

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--purge_orphaned_data BOOL] [--db URI] [--db_import]
                   [--inspect] [--version_tb] [--tag TAG] [--event_file PATH]
                   [--path_prefix PATH] [--window_title TEXT]
                   [--max_reload_threads COUNT] [--reload_interval SECONDS]
                   [--reload_task TYPE] [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [--debugger_data_server_grpc_port PORT]
                   [--debugger_port PORT] [--master_tpu_unsecure_channel ADDR]
                   [--whatif-use-unsafe-custom-prediction YOUR_CUSTOM_PREDICT_FUNCTION.py]
                   [--whatif-data-dir PA

In [12]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
